In [4]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    os.environ['CUDA_VISIBLE_DEVICES'] = \
        os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0]
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


4
4
Sun Nov  5 15:24:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   45C    P0   126W / 300W |   7591MiB / 32510MiB |     94%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/

In [5]:
import os
import json
import argparse
import logging
import random
import numpy as np
import torch
import datasets
from alpaca_eval import evaluate as alpaca_farm_evaluate
from eval.utils import query_openai_chat_model, query_openai_model, generate_completions, dynamic_import_function, load_hf_lm_and_tokenizer

In [167]:
parser = argparse.ArgumentParser()
parser.add_argument("--reference_path", type=str, default="data/eval/alpaca_farm/davinci_003_outputs_2048_token.json", help="Path to the reference outputs. Alpaca_eval leaderboard use davinci_003 to generate the reference outputs, but they limit the max_tokens to 300. Here regenerated reference outputs with max_tokens=2048.")
parser.add_argument("--save_dir", type=str, default="results/alpaca_farm")
parser.add_argument("--model_name_or_path", type=str, default=None, help="If specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="If specified, we will load the tokenizer from here.")
parser.add_argument("--use_slow_tokenizer", action="store_true", help="If given, we will use the slow tokenizer.")
parser.add_argument("--openai_engine", type=str, default=None, help="If specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="Batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="Load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, we will use the chat format for the prompts.")
parser.add_argument("--chat_formatting_function", type=str, default="eval.templates.create_prompt_with_tulu_chat_format", help="The function to use to create the chat format. This function will be dynamically imported. Please see examples in `eval/templates.py`.")
parser.add_argument("--use_vllm", action="store_true", help="If given, we will use vLLM to generate the predictions - much faster.")
parser.add_argument("--annotators_config", type=str, default="alpaca_eval_gpt4_0314")
parser.add_argument("--max_num_examples", type=int, default=None, help="maximum number of examples to evaluate.")


model_name_or_path = '../results/baselines/huggyllama/llama-7b'
model_name_or_path = '../results/ft1_ep=1/llama-7b_tuluv1m'
save_dir = os.path.join(model_name_or_path, 'eval/alpaca_farm/')
model_name_or_path = '../results/oi5_ultrachat:mistral-7b/mistral-7b_ultrachat200k_score=log:prob:neg_pace=prune:size=50000:ep=3'

# eval_batch_size <=5 without oom for llama-7b gen 2048 new tokens

#     --use_chat_format \
#     --max_num_examples 50 \
#     --max_num_examples 5 \
cmd = f"""
    --reference_path alpaca_eval_data \
    --model_name_or_path {model_name_or_path} \
    --save_dir {save_dir} \
    --eval_batch_size 5 \
    --use_chat_format \
    --annotators_config chatgpt \
"""

args = jpt_parse_args(parser, cmd)
args

Namespace(reference_path='alpaca_eval_data', save_dir='../results/ft1_ep=1/llama-7b_tuluv1m/eval/alpaca_farm/', model_name_or_path='../results/oi5_ultrachat:mistral-7b/mistral-7b_ultrachat200k_score=log:prob:neg_pace=prune:size=50000:ep=3', tokenizer_name_or_path=None, use_slow_tokenizer=False, openai_engine=None, eval_batch_size=5, load_in_8bit=False, gptq=False, use_chat_format=True, chat_formatting_function='eval.templates.create_prompt_with_tulu_chat_format', use_vllm=False, annotators_config='chatgpt', max_num_examples=None)

In [168]:

random.seed(42)
os.makedirs(args.save_dir, exist_ok=True)

if args.reference_path != 'alpaca_eval_data':
    raise NotImplementedError('Only support alpaca_eval_data for now. Need to work with `max_num_examples` in case I want to use other reference generation.')

logging.info("loading data and model...")
alpaca_eval_data = datasets.load_dataset("tatsu-lab/alpaca_farm", "alpaca_farm_evaluation")["eval"]

if args.max_num_examples and len(alpaca_eval_data) > args.max_num_examples:
    inds = random.sample(range(len(alpaca_eval_data)), args.max_num_examples)
    alpaca_eval_data = alpaca_eval_data.select(inds)

prompts = []
chat_formatting_function = dynamic_import_function(args.chat_formatting_function) if args.use_chat_format else None
for example in alpaca_eval_data:
    prompt = example["instruction"] + "\n\n" + example["input"] if example["input"] != "" else example["instruction"]
    if args.use_chat_format:
        messages = [{"role": "user", "content": prompt}]
        prompt = chat_formatting_function(messages, add_bos=False)
    prompts.append(prompt)


INFO:root:loading data and model...


  0%|          | 0/1 [00:00<?, ?it/s]

In [181]:
# def format_prompt_template(x):
#     return {
#         'prompt': 
#     }


# ds = alpaca_eval_data.map(format_prompt_template)
# ds
p = '../results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv/eval/alpacafarm_ann=chatgpt_chatfmt/mistral-7b_ultrachat200k_beforesplitlongconv-greedy-long-output.json'
import json
with open(p, 'rb') as f:
    anns = [json.loads(l) for l in f]

prompts = []
for output_1, output_2, instr, inp in zip(
    [x['output'] for x in anns],
    ds['output'],
    ds['instruction'],
    ds['input']):
    
    s = prompt_template.format(
        output_1=output_1, 
        output_2=output_2, 
        instruction=instr + "\n\n" + inp if inp != "" else instr)
    prompts.append(s)
    
print(max(len(x) for x in prompts))

9139


In [184]:
o = tokenizer(prompts)

In [186]:
max([len(x) for x in o.input_ids])

2617

In [187]:
print(prompts[1])

<|im_start|>system
You are a helpful instruction-following assistant.
<|im_end|>
<|im_start|>user
Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here's an example:

# Example:
## Instruction:
Give a description of the following job: "ophthalmologist"

## Output (a):
An ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.

## Output (b):
An ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.

## Which is best, Output (a) or Output (b)?
Output (a)

Here the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.

# Task:
Now is the real task, do not explain your answer, just say Output (a) or Output (b).

## Instruction:
How did

805

In [169]:
model, tokenizer = load_hf_lm_and_tokenizer(
    model_name_or_path=args.model_name_or_path,
    tokenizer_name_or_path=args.tokenizer_name_or_path if args.tokenizer_name_or_path is not None else args.model_name_or_path,
    load_in_8bit=args.load_in_8bit,
    device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
    gptq_model=args.gptq,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
out = tokenizer(alpaca_eval_data['output'])

In [24]:
np.max([len(x) for x in out.input_ids])

383

In [98]:
# args.annotators_config

import pandas as pd
from alpaca_eval import annotators


class MyAnnotator(annotators.PairwiseAnnotator):
    
    

    ### Public methods ###
    def __call__(self, df_to_annotate: pd.DataFrame, **decoding_kwargs) -> pd.DataFrame:
        """Annotates the given examples.

        Parameters
        ----------
        df_to_annotate : pd.DataFrame
            Examples to annotate

        decoding_kwargs :
            Additional arguments to pass to `fn_completions`.
        """
        df_to_annotate = df_to_annotate.copy()  # avoid in place modifications

        if df_to_annotate.empty:
            df_to_annotate[self.annotation_column] = []
            return df_to_annotate

        df_to_annotate = self._preprocess(df_to_annotate)

        # prompts and completions here will not be the same length as the dataframe due to batching
        prompts, df_to_annotate = self._make_prompts(df_to_annotate)

        completions = self.fn_completions(prompts=prompts, **self.completions_kwargs, **decoding_kwargs)

        annotations_to_save, completions_to_save = self._parse_completions(completions=completions["completions"])
        df_to_annotate[self.annotation_column] = annotations_to_save
        if self.completion_column is not None:
            df_to_annotate[self.completion_column] = completions_to_save

        for k, v in completions.items():
            if k != "completions":
                if len(df_to_annotate[self.annotation_column]) == len(v) * self.batch_size:
                    v = [el for el in v for _ in range(self.batch_size)]
                df_to_annotate[k] = v
                if "per_example" in k:
                    df_to_annotate[k] = df_to_annotate[k] / self.batch_size

        df_annotated = self._postprocess(df_to_annotate)

        return df_annotated


    
import alpaca_eval
Annotator = MyAnnotator

model_outputs=model_results
reference_outputs=args.reference_path if args.reference_path != 'alpaca_eval_data' else alpaca_eval_data
annotators_config=args.annotators_config
model_outputs = alpaca_eval.utils.load_or_convert_to_dataframe(model_outputs)
reference_outputs = alpaca_eval.utils.load_or_convert_to_dataframe(reference_outputs)    

# MyAnnotator()

model_outputs = model_outputs[:1]
reference_outputs = reference_outputs[:1]

annotator_kwargs = dict()
annotator = Annotator(annotators_config=annotators_config, **annotator_kwargs)
# annotations = annotator.annotate_head2head(
#     outputs_1=reference_outputs, outputs_2=model_outputs, **annotation_kwargs
# )

INFO:root:Creating the annotator from `chatgpt`.
INFO:root:Saving annotations to `/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json.


<|im_start|>system
You are a helpful instruction-following assistant.
<|im_end|>
<|im_start|>user
Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here's an example:

# Example:
## Instruction:
Give a description of the following job: "ophthalmologist"

## Output (a):
An ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.

## Output (b):
An ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.

## Which is best, Output (a) or Output (b)?
Output (a)

Here the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.

# Task:
Now is the real task, do not explain your answer, just say Output (a) or Output (b).

## Instruction:
{instru

In [99]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/main.py#L133
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/pairwise_evaluator.py#L155
self = annotator
keys_to_merge = None
outputs_1 = reference_outputs
outputs_2 = model_outputs

if keys_to_merge is None:
    keys_to_merge = self.input_keys

keys_to_merge = list(keys_to_merge)


outputs_1 = alpaca_eval.utils.convert_to_dataframe(outputs_1)
outputs_2 = alpaca_eval.utils.convert_to_dataframe(outputs_2)


# find all the columns that are in both
other_same_cols = [k for k in outputs_1.columns if k in outputs_2 and k not in (keys_to_merge + ["output"])]

df_to_annotate = pd.merge(
    outputs_1,
    outputs_2,
    on=keys_to_merge,
    suffixes=("_1", "_2"),
)

for c in other_same_cols:
    # if the columns are the same, we can drop the _2
    if df_to_annotate[c + "_1"].equals(df_to_annotate[c + "_2"]):
        df_to_annotate = df_to_annotate.drop(columns=c + "_2").rename(columns={c + "_1": c})

# if you are taking the cartesian product, you can have undesired duplicates
df_to_annotate = df_to_annotate.drop_duplicates()

if not (len(outputs_1) == len(outputs_2) == len(df_to_annotate)):
    logging.warning(
        f"The length of outputs before and after merge are not the same. We have len(outputs_1)=="
        f"{len(outputs_1)}, len(outputs_2)=={len(outputs_2)}, and len(df_annotated)=={len(df_to_annotate)}."
        f" This means that there are missing examples or duplicates. We are taking a SQL inner join."
    )

df_to_annotate


,instruction,input,output_1,generator_1,sample_mode_1,dataset,datasplit,output_2,generator_2,sample_mode_2
0,What are the names of some famous actors that ...,,Some famous actors that started their careers ...,text-davinci-003,"temp=0.7,top_p=1.0,max_new_tokens=300",helpful_base,eval,Some famous actors that started their careers ...,llama-7b_tuluv1m-greedy-long,"temp=0,max_new_tokens=2048"


In [109]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L135
# out = self.__call__(df_to_annotate, **decoding_kwargs)

to_annotate = df_to_annotate
chunksize = 128
decoding_kwargs = dict()


# note: not ideal potentially doing a lot of dataframe copies. But given that they should be small, ~ok
df_to_annotate = alpaca_eval.utils.convert_to_dataframe(to_annotate)
# make sure primary keys are strings
for c in self.primary_keys:
    df_to_annotate[c] = df_to_annotate[c].astype(str)



all_annotated = []
for df_chunk in alpaca_eval.utils.dataframe_chunk_generator(df_to_annotate, chunksize, tqdm_desc="Annotation chunk"):
    curr_df_to_annotate = self._preprocess(df_chunk)
#     df_annotated = self._annotate(curr_df_to_annotate, **decoding_kwargs)
#     annotated = self._postprocess_and_store_(df_annotated, df_chunk)
#     all_annotated.extend(annotated)
    break

Annotation chunk:   0%|          | 0/1 [00:00<?, ?it/s]


In [122]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L245
self.annotators['chatgpt']
# curr_df_to_annotate = curr_df_to_annotate.drop(columns=['annotator', 'preference', 'price_per_example', 'time_per_example'])
curr_df_to_annotate.loc[0,'annotator'] = 'chatgpt'


curr_idcs = curr_df_to_annotate[self.annotator_column] == 'chatgpt'
# if self.annotation_key in curr_df_to_annotate.columns:
#     curr_idcs &= curr_df_to_annotate[self.annotation_key].isna()


# # actual annotation
# curr_annotated = self.annotators['chatgpt'](
#     curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format],
#     **decoding_kwargs,
# )

In [124]:
curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format]

,instruction,output_1,output_2,annotator
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...,Some famous actors that started their careers ...,chatgpt


In [133]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L547
# SingleAnnotator
df_to_annotate = curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format]


df_to_annotate = df_to_annotate.copy()  # avoid in place modifications

# if df_to_annotate.empty:
#     df_to_annotate[self.annotation_column] = []
#     return df_to_annotate

df_to_annotate = self.annotators['chatgpt']._preprocess(df_to_annotate)

# prompts and completions here will not be the same length as the dataframe due to batching
prompts, df_to_annotate = self.annotators['chatgpt']._make_prompts(df_to_annotate)


In [141]:
prompt_template = self.annotators['chatgpt'].prompt_template
batch_size=self.annotators['chatgpt'].batch_size
prompts, _ = alpaca_eval.utils.make_prompts(df=df_to_annotate, template=prompt_template, batch_size=batch_size)


In [146]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/utils.py#L94
df = df_to_annotate
template = prompt_template
import re
from collections import Counter

text_to_format = re.findall(r"{([^ \s]+?)}", template)
n_occurrences = Counter(text_to_format)
n_occurrences


Counter({'instruction': 1, 'output_1': 1, 'output_2': 1})

In [151]:
import tiktoken
# encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken


ProxyError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Filtered')))

In [32]:
outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts[:5],
    max_new_tokens=2048,
    do_sample=False,
    temperature=0,
    batch_size=args.eval_batch_size if args.eval_batch_size else 1,
)


Generating Completions:   0%|          | 0/5 [00:00<?, ?it/s]/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
Generating Completions: 100%|██████████| 5/5 [01:51<00:00, 22.39s/it]


In [ ]:
for i in range(len(outputs)):
    print(i, )
    prompt_len = len(tokenizer(prompts[i], add_special_tokens=False)['input_ids'])
    print(f'---------- user (length={prompt_len}) ----------')
    print(prompts[i])
    output_len = len(tokenizer(outputs[i], add_special_tokens=True)['input_ids'])
    print(f'---------- assistant (length={output_len}) ----------')
    print(outputs[i])
    print('\n\n\n')
    

In [58]:

model_name = os.path.basename(os.path.normpath(args.model_name_or_path)) if args.model_name_or_path is not None else args.openai_engine
model_results = []
with open(os.path.join(args.save_dir, f"{model_name}-greedy-long-output.json"), "w") as fout:
    for example, output in zip(alpaca_eval_data, outputs):
        example["output"] = output
        example["generator"] = f"{model_name}-greedy-long"
        example["sample_mode"] = "temp=0,max_new_tokens=2048"
        fout.write(json.dumps(example) + "\n")
        model_results.append(example)

In [ ]:
model_

In [ ]:
df_leaderboard, annotations = alpaca_farm_evaluate(
    model_outputs=model_results,
    reference_outputs=args.reference_path if args.reference_path != 'alpaca_eval_data' else alpaca_eval_data,
    annotators_config=args.annotators_config,
    output_path=args.save_dir,
    is_return_instead_of_print=True,
)

In [ ]:

# prices = [x['price_per_example'] for x in annotations]
# print(f'Price (per-example / total) = {np.mean(prices):.4f} / {np.sum(prices):.2f}')

# times = [x['time_per_example'] for x in annotations]
# print(f'Time  (per-example / total) = {np.mean(times):.4f} / {np.sum(times):.2f}')

# df_leaderboard['avg_output_tok_length'] = np.mean(
#     [len(tokenizer(x['output'])['input_ids']) for x in model_results])
# df_leaderboard['price'] = np.sum(prices)

# df_leaderboard.insert(0, 'model', df_leaderboard.index)
df_leaderboard.reset_index(drop=True, inplace=True)

# print(df_leaderboard.to_string(float_format="%.2f"))
display(df_leaderboard)
# save to json

with open(os.path.join(args.save_dir, f"metrics.json"), "w") as fout:
    json.dump(df_leaderboard.iloc[0].to_dict(), fout)

In [ ]:
df_leaderboard